## Import

In [1]:
import pytorch_lightning as pl
import sys 
sys.path.append('/app/')
RANDOM_SEED = 42
pl.seed_everything(seed=RANDOM_SEED)
print("done")

Global seed set to 42


done


# Segmentation baseline

### The LSTM can be replaced by Transformer

In [2]:
import logging
logging.disable(logging.CRITICAL)

exp_args = {
    "encoder": "root-interval",
    # "embedding_model": "fasttext",
    "embedding_model_path": "/app/out/root_interval_best/root-interval-fasttext-with-Processed-ChoCo.ckpt"
}

segmentation_train_args = {
    "seed": 42,
    "test_mode" : False, # If test_mode = true, then we use 3 track for test
    "full_chord": False,
    "use_pitchclass2vec_model": False if exp_args.get("embedding_model") == None else True,
    "disable_wandb" : False,
    "num_labels" : 11,
    # "embedding_dim" : None,  # Default as None，will use p2v.vector_size
    "hidden_size" : 100,
    "num_layers" : 5,
    "dropout" : 0.5,
    "learning_rate" : 0.001,
    "batch_size": 128,
    "max_epochs": 1,
    "factor": 0.5,
    "patience": 5,

    "wandb_run_name" : "test_run",
    "out": "/app/segmentation_out",

}

print(segmentation_train_args.get("use_pitchclass2vec_model"))
print("done")

False
done


In [3]:
# Auto generate a Linux command
command_parts = ["python /app/tasks/segmentation/segmentation_train.py"]

for arg, value in exp_args.items():
    command_parts.append(f"--{arg} {value}")

for arg, value in segmentation_train_args.items():
    command_parts.append(f"--{arg} {value}")

command = " ".join(command_parts)
print(command)

print("done!")

python /app/tasks/segmentation/segmentation_train.py --encoder root-interval --embedding_model_path /app/out/root_interval_best/root-interval-fasttext-with-Processed-ChoCo.ckpt --seed 42 --test_mode False --full_chord False --use_pitchclass2vec_model False --disable_wandb False --num_labels 11 --hidden_size 100 --num_layers 5 --dropout 0.5 --learning_rate 0.001 --batch_size 128 --max_epochs 1 --factor 0.5 --patience 5 --wandb_run_name test_run --out /app/segmentation_out
done!


In [4]:
# Excecute the Linux command
!{command}
print("done!")

wandb: Currently logged in as: cretaceousmart. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /app/tasks/segmentation/wandb/run-20231110_162946-bahwnpvx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run test_run
wandb: ⭐️ View project at https://wandb.ai/cretaceousmart/pitchclass2vec_Segmentation
wandb: 🚀 View run at https://wandb.ai/cretaceousmart/pitchclass2vec_Segmentation/runs/bahwnpvx
100%|████████████████████████████████████████| 890/890 [00:01<00:00, 850.93it/s]
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /app/segmentation_out exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Sanity Checking: 0it [00:00, ?it/s]Jie log: len(chords): 210, len(labels)